# Gradio

Learn more: https://www.gradio.app/

In [ ]:
import os, sys
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
import gradio as gr

In [ ]:
load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY')
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

if not OPENAI_API_KEY:
    print("OPENAI_API_KEY not loaded.")
    sys.exit(1)
if not ANTHROPIC_API_KEY:
    print("ANTHROPIC_API_KEY not loaded.")
    sys.exit(1)
if not GEMINI_API_KEY:
    print("GEMINI_API_KEY not loaded.")
    sys.exit(1)

In [ ]:
openai = OpenAI()
claude = anthropic.Anthropic()
google.generativeai.configure()

In [ ]:
def message_gpt(prompt):
    completion = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": prompt}
        ]
    )
    return completion.choices[0].message.content

In [ ]:
message_gpt("Who is currently the president of USA?")

In [ ]:
def shout(text: str):
    return text.upper()

In [ ]:
view = gr.Interface(
    fn = shout,
    inputs = [
        gr.Textbox(label = "Your message", placeholder = "Your message", lines = 6)
    ],
    outputs = [
        gr.Textbox(label = "Response", placeholder = "Response", lines = 8)
    ],
    flagging_mode = "never",
    css = "footer{display:none !important}"
)
# view.launch(share=True) # To create a publicly available URL for gradio
view.launch()

In [ ]:
view = gr.Interface(
    fn = message_gpt,
    inputs = [
        gr.Textbox(label = "Your message", placeholder = "Your message", lines = 6)
    ],
    outputs = [
        gr.Textbox(label = "Response", placeholder = "Response", lines = 8)
    ],
    flagging_mode = "never",
    css = "footer{display: none !important}"
)
view.launch()

In [ ]:
view = gr.Interface(
    fn = message_gpt,
    inputs = [
        gr.Textbox(label = "Your message", placeholder = "Your message", lines = 6)
    ],
    outputs = [
        gr.Markdown(label = "Reponse")
    ],
    flagging_mode = "never",
    css = "footer{display: none !important}"
)
view.launch()

In [ ]:
def stream_gpt(prompt):
    stream = openai.chat.completions.create(
        model = "gpt-4o-mini",
        messages = [
            {"role": "system", "content": "You are a helpful assistant"},
            {"role": "user", "content": prompt}
        ],
        stream = True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result
    return result

In [ ]:
view = gr.Interface(
    fn = stream_gpt,
    inputs = [
        gr.Textbox(label = "Your message", placeholder = "Your message", lines = 6)
    ],
    outputs = [
        gr.Markdown(label = "Reponse")
    ],
    flagging_mode = "never",
    css = "footer{display: none !important}"
)
view.launch()

In [ ]:
def stream_claude(prompt):
    result = claude.messages.stream(
        model = "claude-3-haiku-20240307",
        max_tokens = 500,
        temperature = 0.7,
        system = "You are a helpful assistant",
        messages = [
            {"role": "user", "content": prompt}
        ]
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response
    return response

In [ ]:
view = gr.Interface(
    fn = stream_claude,
    inputs = [
        gr.Textbox(label = "Your message", placeholder = "Your message", lines = 6)
    ],
    outputs = [
        gr.Markdown(label = "Reponse")
    ],
    flagging_mode = "never",
    css = "footer{display: none !important}"
)
view.launch()

In [ ]:
def stream_gemini(prompt):
    model = google.generativeai.GenerativeModel("gemini-2.5-flash", system_instruction = "You are a helpful assistant")
    response_stream = model.generate_content(
        prompt,
        stream = True,
        generation_config = google.generativeai.GenerationConfig(
            temperature = 0.7,
            max_output_tokens = 500
        )
    )
    response = ""
    for chunk in response_stream:
        response += chunk.text or ""
        yield response
    return response

In [ ]:
def stream_model(prompt, model):
    if model.startswith("GPT"):
        result = stream_gpt(prompt)
    elif model.startswith("Claude"):
        result = stream_claude(prompt)
    elif model.startswith("Gemini"):
        result = stream_gemini(prompt)
    else:
        raise ValueError("Unsupported model type")
    for chunk in result:
        yield chunk

In [ ]:
view = gr.Interface(
    fn = stream_model,
    inputs = [
        gr.Textbox(label = "Your message", placeholder = "Your message", lines = 6),
        gr.Dropdown(["GPT-4", "Claude", "Gemini"], label="Select Model", value="GPT-4")
    ],
    outputs = [
        gr.Markdown(label = "Reponse")
    ],
    flagging_mode = "never",
    css = "footer{display: none !important}"
)
view.launch()